In [1]:
import torch_geometric as tg

from models.se3_transformer import Se3EquivariantTransformer
from utils.load_md17 import load_md17
import e3nn

import torch
import torch_geometric as tg
import pytorch_lightning as pl
import torchmetrics


In [2]:
class MD17TransformerTask(pl.LightningModule):
    def __init__(self, energy_model: torch.nn.Module, lr=1e-3, force_loss_weight=500):
        super().__init__()
        self.energy_model = energy_model
        self.lr = lr
        self.force_loss_weight = force_loss_weight

        self.energy_train_metric = torchmetrics.MeanAbsoluteError()
        self.energy_valid_metric = torchmetrics.MeanAbsoluteError()
        self.energy_test_metric = torchmetrics.MeanAbsoluteError()
        self.force_train_metric = torchmetrics.MeanAbsoluteError()
        self.force_valid_metric = torchmetrics.MeanAbsoluteError()
        self.force_test_metric = torchmetrics.MeanAbsoluteError()

    @staticmethod
    def compute_energy_normalisers(dataset):
        sum_energies = 0
        total_nodes = 0
        force_scales = 0

        for graph in dataset:
            total_nodes += graph.num_nodes
            sum_energies += graph.energy
            force_scales += torch.linalg.vector_norm(graph.force, dim=1).sum()

        mean = sum_energies / total_nodes
        std = force_scales / total_nodes

        return mean, std

    def forward(self, graph):
        # graph.pos = torch.autograd.Variable(graph.pos, requires_grad=True)
        predicted_energy = self.energy_model(graph).sum(1)
        expected_outputs = torch.ones_like(predicted_energy, requires_grad=True)
        predicted_force = -1 * torch.autograd.grad(predicted_energy,
                                                   graph.pos,
                                                   grad_outputs=expected_outputs,
                                                   create_graph=True,
                                                   retain_graph=True,
                                                   # allow_unused=True
                                                   )[0]

        # predicted_energy = predicted_energy.squeeze(-1)

        return predicted_energy, predicted_force

    def energy_and_force_loss(self, graph, energy, force):
        energy_loss = torch.nn.functional.mse_loss(energy, graph.energy)
        force_loss = torch.nn.functional.mse_loss(force, graph.force)
        loss = energy_loss + self.force_loss_weight * force_loss
        return loss

    def training_step(self, graph):
        energy, force = self(graph)
        loss = self.energy_and_force_loss(graph, energy, force)
        self.energy_train_metric(energy, graph.energy)
        self.force_train_metric(force, graph.force)

        cur_lr = self.trainer.optimizers[0].param_groups[0]["lr"]
        self.log("lr", cur_lr, prog_bar=True, on_step=True)
        return loss

    def on_train_epoch_end(self):
        self.log("Energy train MAE", self.energy_train_metric, prog_bar=True)
        self.log("Force train MAE", self.force_train_metric, prog_bar=True)

    @torch.inference_mode(False)
    def validation_step(self, graph, batch_idx):
        energy, force = self(graph)
        self.energy_valid_metric(energy, graph.energy)
        self.force_valid_metric(force, graph.force)

    def on_validation_epoch_end(self):
        self.log("Energy valid MAE", self.energy_valid_metric, prog_bar=True)
        self.log("Force valid MAE", self.force_valid_metric, prog_bar=True)

    def test_step(self, graph, batch_idx):
        energy, force = self.forward(graph)
        self.energy_test_metric(energy, graph.energy)
        self.force_test_metric(force, graph.force)

    def on_test_epoch_end(self):
        self.log("Energy test MAE", self.energy_test_metric, prog_bar=True)
        self.log("Force test MAE", self.force_test_metric, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)

        num_steps = self.trainer.estimated_stepping_batches
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_steps)

        lr_scheduler_config = {
            "scheduler": scheduler,
            "interval": "step",
            "frequency": 1,
        }
        return [optimizer], [lr_scheduler_config]

In [3]:
### Prototype workflow

In [4]:
energy_model = Se3EquivariantTransformer.construct_from_number_of_channels_and_lmax(num_channels=8,
                                                                             l_max=2,
                                                                             num_features=9,
                                                                             num_attention_layers=4,
                                                                             num_attention_heads=4,
                                                                             radial_network_hidden_units=32
                                                                             )

C:\Users\niksm\Documents\CodeForUni\venvs\transformer_invariants\lib\site-packages\torch\jit\_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


In [5]:
batch_size=2
radius=2

data = load_md17(dataset_name='aspirin CCSD', dataset_dir='../real_datasets', radius=radius)

train_dataloader = tg.loader.DataLoader(data['train'], batch_size=batch_size, shuffle=True)
validation_dataloader = tg.loader.DataLoader(data['validation'], batch_size=batch_size)
test_dataloader = tg.loader.DataLoader(data['test'], batch_size=batch_size)

In [6]:
MD17TransformerTask.compute_energy_normalisers(dataset=data['train'])

setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting
setting


(tensor([-837711.3125]), tensor(2015.1858))

In [7]:
trainer = pl.Trainer(max_epochs=1)
task = MD17TransformerTask(energy_model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\niksm\Documents\CodeForUni\venvs\transformer_invariants\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(


In [8]:
for batch in train_dataloader:
    batch = batch.clone()
    break

energy = energy_model(batch)

setting
setting


In [9]:
energy.sum(1)

tensor([0.1483, 0.1499], grad_fn=<SumBackward1>)

In [10]:
trainer.fit(task, train_dataloader, test_dataloader)

Loading `train_dataloader` to estimate number of stepping batches.
C:\Users\niksm\Documents\CodeForUni\venvs\transformer_invariants\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(

  | Name                | Type                      | Params
------------------------------------------------------------------
0 | energy_model        | Se3EquivariantTransformer | 1.2 K 
1 | energy_train_metric | MeanAbsoluteError         | 0     
2 | energy_valid_metric | MeanAbsoluteError         | 0     
3 | energy_test_metric  | MeanAbsoluteError         | 0     
4 | force_train_metric  | MeanAbsoluteError         | 0     
5 | force_valid_metric  | MeanAbsoluteError         | 0     
6 | fo

Sanity Checking: 0it [00:00, ?it/s]

C:\Users\niksm\Documents\CodeForUni\venvs\transformer_invariants\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


setting
setting


RuntimeError: One of the differentiated Tensors does not require grad

In [ ]:
task(batch)[1]

In [ ]:
batch.pos

In [11]:
task.energy_model(batch).requires_grad

True

In [12]:
task.forward(batch)

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.